Find this notebook and other ons [in2p3 gitlab](https://https://gitlab.in2p3.fr/fbi-data/omero-quay/-/tree/dev/notebooks)



## Authors: 

The fbi-omero project

- [Théo Barnouin](https://orcid.org/0009-0008-6583-5806)
- [Emmanuel Faure](https://orcid.org/0000-0003-2787-0885)
- [Guillaume Gay](https://orcid.org/0000-0002-5444-5246)
- [Jean-François Guillaume](https://orcid.org/0009-0009-3475-447X)
- [Marc Mongy](https://orcid.org/0009-0000-8418-5994)
- [Guillaume Maucort](https://orcid.org/0009-0000-9454-4161)
- [Perrine Paul-Giloteaux](https://orcid.org/0000-0002-4822-165X)


## Acknowledgements:

- MRI, ISDM-MESO, GiCiD
- The French and International Openmicroscopy and OpenData communities
- In particular:
    - Josh Moore and all @ ome, quarep, and *BioImaging
    - Julio Mateos-Langerak
    - Olivier Miquel

....

# Import routes demo


## Docker setup

### Starting up docker containers

Run the code bellow from a terminal in a `quay` environement. It takes some time

Please refer to [`docker_compose.yml`](../tests/containers/docker-compose.yml) for a detailed view on the docker volume mappings.


This takes a few minutes on my machine, and resets the import environement. I prefer to execute them step wise to browse the logged data.


```sh
cd .. 
echo 'reseting containers and volumes' && make docker_purge 

make docker_up && echo 'ran docker_up'
make docker_populate && echo 'ran docker_populate'
```

### Monitor the containers with `docker exec`


```sh

docker exec -it  omero-server bash -c "tail -f /opt/omero/server/log/omero_quay/main.log"

# Facilty logs
docker exec -it  quay-facility bash -c "tail -f /home/mambauser/log/omero_quay/main.log"

# Search for tifs Facilty paths
docker exec -it  quay-facility bash -c "ls -lRth /mnt/user/home/facility0/S-JCBD-200902093 | grep tif | cut -d ':' -f 2"
docker exec -it  quay-facility bash -c "ls -lRth /mnt/coop/ | grep tif # | cut -d ':' -f 2"

# search for tif data objects in irods

docker exec -it -u irods irods-icat bash -c "ils -r /tempZone/home/JCB2009 | grep tif" 
```



## Simulated Quay Network


This notebooks and the docker compose it runs emulates a "small" omero-quay network:

An omero-quay network is an implementation of the 
[Bioimage Town concept from Katy Wolstencroft, Josh Moore, Matthew Hartley](https://forum.image.sc/t/welcome-to-bioimage-town/82998)


<img src="../docs/images/docker_architecture.svg" width="800">


A **store** is where data and metadata are written and updated. 

**Stores** are grouped in **Provenances**. Here we simulate a **facility** buffer provenance (lower) and a **mesocenter** provenance (upper). Each store has a **scheme**, either **`file`**, **`irods`** or **`omero`**, depending on its interface to the data. A store either respects the ISA hierarchy or not. 

<img src="../docs/images/docker_architecture_keys.svg" width="800">

Data and metadata are tracked through a **Manifest**, going through a **Route** between stores.

**Clerks** take a manifest and the associated data from store to strore; a clerck is a scheme specific process calling the relevant API's `parse`, `create`, `update`, `transfer` and `delete` methods.

For example, from an iRODS store to an  Omero store, the omero clerck might update the `ome.xml[.jsonld]` or  use `omero-cli-transfer` to import data `inplace` in the Omero serveur. Quay **services** at each provenance listens through a tornado server. Post request trigger Clerks, while Get requets trigger access to the Mongo DB registering the manifests.



## Example of a route from facility to the omero-server / icat containers :


This is a buzy map, but when we focus on a single route, it is not that complicated :)



We want to go from the most local data source (lower left) to the meso center OMERO serveur (center). Data annotation is done through the excel file.


<hr/>

<img src="../docs/images/docker_architecture_single_route.png" width="800">

<hr/>



A simpler, linear view of our route:

<img src="../docs/images/facilty_to_docker.svg" width="720">


We do all the packaging work on the facility machine. As it acts as a buffer, it makes sense. 

On this route, we conform early to the ISA hierarchy. This offloads the central server and allows for a faster feed-back on the import process.   


So this is route **f1-f2-fo-f2i-mo-f2i**, from facilityUserFile **f1** to mesoOmero, **mo**, via faciltyCoopFile (**fo**), faciltyOmero (**fo**) and facilityCoopResc (**f2i**).

Some store (those related to iRODS) are visited twice to fully update their metadata post-import.

Before we start our little trip, some house keeping:

